# Realizacja Konkursu Kaggle 

## Gromadzenie i analiza danych o użytkownikach mobilnych

## Implementacja

In [2]:
# biblioteki do analizy danych
import pandas as pd
import numpy as np

# biblioteki do wizualizacji danych
import matplotlib.pyplot as plt
import seaborn as sns

# biblioteki do uczenia maszynowego
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

source = 'E:\Kaggle\TSI\konkurs kaggle - data/'

In [3]:
# wczytanie danych
app_events = pd.read_csv(source+'app_events.csv', sep=',')
app_labels = pd.read_csv(source+'app_labels.csv', sep=',')
events = pd.read_csv(source+'events.csv', sep=',')
gender_age_test = pd.read_csv(source+'gender_age_test.csv', sep=',')
gender_age_train = pd.read_csv(source+'gender_age_train.csv', sep=',')
label_categories = pd.read_csv(source+'label_categories.csv', sep=',')
phone_brands = pd.read_csv(source+'phone_brands.csv', sep=',')

### 1) Analiza cech 

#### Analiza label_categories
Analizowana jest zmieniona wersja label_categories - usunięte zostały najmniej popularne kategorie istniejące łączeni na około 20% różnych urządzeń. Takie rozwiązanie zmniejszyło liczbę kategorii z 450 różnych, do dokładnie 107 najpopularniejszych, co korzystnie wpłynie na czas wykonywania obliczeń.

In [7]:
# Liczba wypełnionych wierszy danej kolumny
label_categories.count()

label_id    107
category    107
dtype: int64

In [8]:
# Liczba pustych wierszy danej kolumny
label_categories.isnull().sum()

label_id    0
category    0
dtype: int64

Liczba powtórzeń konkretnej kategorii - występuje łącznie 7 powtórzeń, uzyskany wektor One-Hot powinien zatem mieć
dokładnie 107-7=100 kolumn + jedna przeznaczona na label_id

In [6]:
#Wyświetlenie powtarzających się kategorii
label_categories_group = label_categories.groupby('category')['label_id']
label_categories_group.nunique().sort_values(ascending=False)

category
unknown                              4
Pay                                  2
mobile bank                          2
Personal Effectiveness               2
P2P                                  2
Accounting                           1
Irritation / Fun 1                   1
High-speed rail train reservation    1
Higher income                        1
IM                                   1
IMF                                  1
Industry tag                         1
Integrated Air Travel Booking        1
Integrated Living                    1
Internet Banking                     1
Internet banking                     1
Jobs                                 1
Air Travel                           1
Liquid medium                        1
Low Risk                             1
Low income                           1
Low liquidity                        1
Low profitability                    1
Low risk                             1
Moderate profitability               1
Monetary Fund   

In [9]:
label_categories.head()

,label_id,category
0,1003,fashion
1,1007,P2P net loan
2,1008,Pay
3,1012,Internet Banking
4,1014,Consumer Finance


In [10]:
#Ostatecznie jest 107 kategorii o różnych ID
label_categories.shape

(107, 2)

In [11]:
# wygenerowanie wektora One-Hot
df = pd.get_dummies(label_categories['category'])

In [12]:
# doklejenie label_id do wektora One-Hot, w celu późniejszego zmergowania tak powstałej tabeli z app_label
frames = [label_categories.label_id, df]
result = pd.concat(frames, axis=1)

In [13]:
# weryfikacja, czy uzyskaliśmy dokładnie taki rozmiar jakiego oczekiwaliśmy.
result.shape

(107, 101)

In [14]:
# poglądowe wyświetlenie wyniku
result.head()

,label_id,1 free,1 reputation,A shares,Accounting,Air Travel,And the Church,Bank Credit Card,Bank financing,Booking channels,...,reading platform,service,stock,the film,tourism product,train,travel,unknown,video,weibo
0,1003,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1007,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1008,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1012,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1014,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2) Mergowanie danych
Mergowany jest wektor one-hot nazw kategorii, wraz z tabelą app_labels

In [15]:
# merge tabeli app_labels + label_categories
AppLabels_LabelCategories = pd.merge(app_labels, result, how="inner")
# usunięcie kolumny label_id - nie będzie już potrzebna w przyszłości
AppLabels_LabelCategories = AppLabels_LabelCategories.drop('label_id', axis=1)
# usunięcie duplikatów
AppLabels_LabelCategories = AppLabels_LabelCategories.drop_duplicates()
# zapisanie do pliku
AppLabels_LabelCategories.to_csv(source + 'merge/AppLabels_LabelCategories.csv', index=False)

Przygotowanie tabeli app_events do mergowania - usunięta zostanie kolumna is_installed, oraz wszystkie wiersze dla których is_active = 0

In [15]:
# wczytanie pliku
app_events = pd.read_csv(source+'app_events.csv', sep=',')
# usunięcie kolumny is_installed
app_events = app_events.drop('is_installed',axis=1)
# usunięcie wierszy dla których is_active = 0
app_events = app_events.drop(app_events[app_events.is_active == 0].index)

,event_id,app_id,is_active
0,2,5927333115845830913,1
1,2,-5720078949152207372,0
2,2,-1633887856876571208,0
3,2,-653184325010919369,1
4,2,8693964245073640147,1


In [20]:
#zapisanie app_events do nowego pliku
app_events.to_csv(source + 'merge/app_events_cleaned.csv', index=False)

In [22]:
#upewnienie się że wszystkie pola są wypełnione
app_events.isnull().sum()
#sprawdzenie typów
app_events.dtypes

event_id     int64
app_id       int64
is_active    int64
dtype: object

In [2]:
#wczytanie pliku AppLabels_LabelCategories.csv
AppLabels_LabelCategories = pd.read_csv(source+'merge/AppLabels_LabelCategories.csv', sep=',')
# merge tabeli app_events + AppLabels_LabelCategories - wykorzystanie podziału na bloki ze względu na problem z pamięcią
app_events = pd.read_csv(source+'merge/app_events_cleaned.csv', sep=',', iterator=True, chunksize=700000)
# zastosowanie flagi jest niezbędne w celu uniknięcia przepisywania nazw kolumn przy każdym bloku
flag = True
for chunk in app_events:
    #mergowanie tablic
    chunk = pd.merge(chunk, AppLabels_LabelCategories, how="inner")
    #usunięcie kolumny is_active oraz app_id, gdyż teraz są one zbędne
    chunk = chunk.drop(['app_id', 'is_active'], axis=1)
    #grupowanie po event_id, reset_index() powoduje zapisywanie event_id (normalnie było ono ignorowane i niezapisywane)
    chunk = chunk.groupby("event_id").sum().reset_index()
    #zapisanie do pliku
    chunk.to_csv(source + 'merge/AppEvents_AppLabels_LabelCategories.csv', index=False, mode='a', header=flag)
    flag= False

In [14]:
#Upewnienie się, czy liczba event_id jest taka sama jak w oryginalnym pliku
AppEvents_AppLabels_LabelCategories = pd.read_csv(source+'merge/AppEvents_AppLabels_LabelCategories.csv', sep=',')
AppEvents_AppLabels_LabelCategories.shape

(1471839, 101)

In [15]:
#W oryginalnym pliku jest ~6tyś więcej unikatowych event_id, co jest niewielkim odchyleniem 
app_events_cleaned = pd.read_csv(source+'merge/app_events_cleaned.csv', sep=',')
app_events_cleaned = app_events_cleaned.drop(['app_id','is_active'], axis=1)
app_events_cleaned = app_events_cleaned.drop_duplicates()
app_events_cleaned.shape

(1477059, 1)

Ze względu na fakt, że podczas merge'a i grupowania plik był dzielony na bloki, mogło dojść do sytuacji w której blok który się kończył, posiadał ten sam event_id co nowo rozpoczęty blok. Aby usunąć ryzyko takiego pominięcia, raz jeszcze pogrupuję plik - tym razem bez podziału na bloki, gdyż jest on dużo mniejszy.

In [17]:
#wczytanie ostatnio zmergowanego pliku
grupowane = pd.read_csv(source+'merge/AppEvents_AppLabels_LabelCategories.csv', sep=',')
#grupowanie po event_id, reset_index() powoduje zapisywanie event_id (normalnie było ono ignorowane i niezapisywane)
grupowane = grupowane.groupby("event_id").sum().reset_index()
#zapisanie do pliku
grupowane.to_csv(source + 'merge/AppEvents_AppLabels_LabelCategories_cleaned.csv', index=False)

In [22]:
#ponowne porównanie liczby eventów - Okazuje się, że 15 event_id zostało pominiętych
grupowane.shape

(1471824, 101)

### 3) Przygotowywanie zbioru treningowego:

Wczesniej przygotowany został plik Phone_Train_Events_Medians.csv, zawierający zmergowane następujące tabele:

phone_brands - zmienione zostały nazwy modeli i marek na int'y, 
gender_age_train - postać oryginalna,
events - powtarzające się wartości dla device_id zostały zastąpione medianą, dzięki czemu uzyskano postać bez powtórzeń

In [13]:
Phone_Train_Events_Medians = pd.read_csv(source+'merge/Phone_Train_Events_Medians.csv', sep=',')
AppEvents_AppLabels_LabelCategories_cleaned = pd.read_csv(source+'merge/AppEvents_AppLabels_LabelCategories_cleaned.csv', sep=',')

#Aby mieć pełen plik, należy zmergować plik AppEvents_AppLabels_LabelCategories_cleaned.csv z Phone_Train_Events_Medians.csv
Train = pd.merge(Phone_Train_Events_Medians, AppEvents_AppLabels_LabelCategories_cleaned, how="left")
#Należy usunąć kolumne event_id
Train = Train.drop('event_id',axis=1)
#Problemem jest fakt, że po zapisaniu w takiej postaci, istnieje wiele powtórzeń device_id które wynikają z event_id. 
#Finalnie chcemy uzyskać postać w której każde device_id będzie istnieć tylko raz.
#Najpierw puste pola wypełnione zostaną zerami
Train = Train.fillna(0)
Train.to_csv(source + 'merge/Train.csv', index=False)

In [2]:
#Uzyskany plik
Train = pd.read_csv(source+'merge/Train.csv', sep=',')
Train.head()

,device_id,group,phone_brand,device_model,longitude,latitude,hour,1 free,1 reputation,A shares,...,reading platform,service,stock,the film,tourism product,train,travel,unknown,video,weibo
0,9032155484127182494,M29-31,4,284,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,8026504930081700361,M23-26,1,263,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,8026504930081700361,M23-26,1,263,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,8026504930081700361,M23-26,1,263,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,2313145512701915151,M32-38,2,5,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Aby każde device wystąpiło tylko raz, usunięte zostaną wszystkie kolumny od 'group' aż do 'hour',
następnie plik zostanie pogrupowany według device_id i zapisany.
Zapisany plik zmerguje ponownie z Phone_Train_Events_Medians.csv

In [3]:
#Wczytanie pliku
Train = pd.read_csv(source+'merge/Train.csv', sep=',')
#Usunięcie kolumn które uniemożliwiają zsumowanie
Train = Train.drop(['group','phone_brand','device_model','longitude','latitude','hour'],axis=1)
#Grupowanie po device_id
Train = Train.groupby("device_id").sum().reset_index()
Train.to_csv(source + 'merge/Train_cleaned.csv', index=False)

In [4]:
Train.shape

(74645, 101)

In [8]:
#Porównanie liczby device_id pliku train_cleaned.csv z liczbą device_id pliku gender_age_train.csv
gender_age_train = pd.read_csv(source+'gender_age_train.csv', sep=',')
gender_age_train.drop(['group','gender','age'], axis=1)
gender_age_train.drop_duplicates()
gender_age_train.shape

(74645, 4)

In [7]:
#Wczytanie plików które chcemy ostatecznie zmergować
Phone_Train_Events_Medians = pd.read_csv(source+'merge/Phone_Train_Events_Medians.csv', sep=',')
Phone_Train_Events_Medians = Phone_Train_Events_Medians.drop('event_id', axis=1)
Train_cleaned = pd.read_csv(source+'merge/Train_cleaned.csv', sep=',')

#Mergowanie
Train_Set = pd.merge(Phone_Train_Events_Medians, Train_cleaned, how="left")
#Zapisanie do pliku
Train_Set.to_csv(source + 'merge/Train_Set.csv', index=False)

In [9]:
#Train_Set jest już prawie gotowy, potrzeba jeszcze usunąć duplikaty które pozostały po poprzednim mergu
Train_Set = Train_Set.drop_duplicates()
Train_Set.to_csv(source + 'merge/Train_Set_cleaned.csv', index=False)

In [10]:
#Jak widać liczba device_id jest o 1 większa niż w gender_age_train.csv, najprawdopodobniej istnieje device_id które zostało pominięte
Train_Set.shape

(74646, 107)

In [11]:
#Podgląd tabeli
Train_Set.head()

,device_id,group,phone_brand,device_model,longitude,latitude,hour,1 free,1 reputation,A shares,...,reading platform,service,stock,the film,tourism product,train,travel,unknown,video,weibo
0,9032155484127182494,M29-31,4,284,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,8026504930081700361,M23-26,1,263,0,0,0,0,0,0,...,0,0,0,0,0,0,0,3,0,0
4,2313145512701915151,M32-38,2,5,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,-7303294878482532208,M32-38,4,174,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,3014072209520807550,M27-28,6,269,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Udało się uzyskać zbiór treningowy, teraz należy w podobny sposób przygotować zbiór testowy

### 4) Przygotowywanie zbioru testowego:

In [13]:
Phone_Test_Events_Medians = pd.read_csv(source+'merge/Phone_Test_Events_Medians.csv', sep=',')
AppEvents_AppLabels_LabelCategories_cleaned = pd.read_csv(source+'merge/AppEvents_AppLabels_LabelCategories_cleaned.csv', sep=',')

#Aby mieć pełen plik, należy zmergować plik AppEvents_AppLabels_LabelCategories_cleaned.csv z Phone_Train_Events_Medians.csv
Test = pd.merge(Phone_Test_Events_Medians, AppEvents_AppLabels_LabelCategories_cleaned, how="left")
#Należy usunąć kolumne event_id
Test = Test.drop('event_id',axis=1)
#Problemem jest fakt, że po zapisaniu w takiej postaci, istnieje wiele powtórzeń device_id które wynikają z event_id. 
#Finalnie chcemy uzyskać postać w której każde device_id będzie istnieć tylko raz.
#Najpierw puste pola wypełnione zostaną zerami
Test = Test.fillna(0)
Test.to_csv(source + 'merge/Test.csv', index=False)

In [3]:
#Uzyskany plik
Test = pd.read_csv(source+'merge/Test.csv', sep=',')
Test.head()

,device_id,phone_brand,device_model,longitude,latitude,hour,1 free,1 reputation,A shares,Accounting,...,reading platform,service,stock,the film,tourism product,train,travel,unknown,video,weibo
0,-6220210354783429585,4,74,0,0,7,2,0,1,0,...,1,0,1,0,0,0,0,2,3,0
1,-6220210354783429585,4,74,0,0,7,1,0,1,0,...,1,0,1,0,0,0,0,2,2,0
2,-6220210354783429585,4,74,0,0,7,1,0,2,0,...,0,0,2,0,0,0,0,3,2,0
3,-6220210354783429585,4,74,0,0,7,2,0,1,0,...,2,0,1,0,0,0,0,2,3,0
4,-6220210354783429585,4,74,0,0,7,1,0,1,0,...,2,0,1,0,0,0,0,2,2,0


In [4]:
#Usunięcie kolumn które uniemożliwiają zsumowanie
Test = Test.drop(['phone_brand','device_model','longitude','latitude','hour'],axis=1)
#Grupowanie po device_id
Test = Test.groupby("device_id").sum().reset_index()
#Zapisanie wyniku do pliku
Test.to_csv(source + 'merge/Test_cleaned.csv', index=False)

In [5]:
Test.shape

(112071, 101)

In [16]:
#Porównanie liczby device_id pliku test_cleaned.csv z liczbą device_id pliku gender_age_test.csv
gender_age_test = pd.read_csv(source+'gender_age_test.csv', sep=',')
gender_age_test.shape

(112071, 1)

In [9]:
#Wczytanie plików które chcemy ostatecznie zmergować
Phone_Test_Events_Medians = pd.read_csv(source+'merge/Phone_Test_Events_Medians.csv', sep=',')
Phone_Test_Events_Medians = Phone_Test_Events_Medians.drop('event_id', axis=1)
Test_cleaned = pd.read_csv(source+'merge/Test_cleaned.csv', sep=',')

#Mergowanie
Test_Set = pd.merge(Phone_Test_Events_Medians, Test_cleaned, how="left")
#Zapisanie do pliku
Test_Set.to_csv(source + 'merge/Test_Set.csv', index=False)

In [10]:
#Test_Set jest już prawie gotowy, potrzeba jeszcze usunąć duplikaty które pozostały po poprzednim mergu
Test_Set = Test_Set.drop_duplicates()
Test_Set.to_csv(source + 'merge/Test_Set_cleaned.csv', index=False)

In [11]:
#Jak widać liczba device_id jest o 5 większa niż w gender_age_test.csv, najprawdopodobniej istnieje device_id które zostało pominięte
Test_Set.shape

(112076, 106)

In [27]:
#W celu wyrównania liczby device_id, sprawdziłem id które się powtarzają i usunąłem je ręcznie w notepadzie
Test['device_id'].value_counts().sort_values(ascending=False).head(6)

-7297178577997113203    2
-5269721363279128080    2
-7059081542575379359    2
-6590454305031525112    2
-3004353610608679970    2
 3455183851679931327    1
Name: device_id, dtype: int64

In [28]:
Train['device_id'].value_counts().sort_values(ascending=False).head(3)

 5245428108336915020    2
-2148167032341924228    1
-3435121045444177990    1
Name: device_id, dtype: int64

In [12]:
#Podgląd tabeli
Test_Set.head()

,device_id,phone_brand,device_model,longitude,latitude,hour,1 free,1 reputation,A shares,Accounting,...,reading platform,service,stock,the film,tourism product,train,travel,unknown,video,weibo
0,-6220210354783429585,4,74,0,0,7,15,0,10,0,...,11,0,10,0,0,0,0,19,24,0
9,289797889702373958,1,53,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,5751283639860028129,2,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11,6261900729835972204,2,3,104,30,11,1,0,0,0,...,0,0,0,0,0,0,0,1,1,0
15,3790071869532760430,1,263,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 5) Uczenie maszynowe 

In [32]:
#Wczytanie plików
Train = pd.read_csv(source+'merge/Train_Set_cleaned.csv', sep=',')
Test = pd.read_csv(source+'merge/Test_Set_cleaned.csv', sep=',')

In [41]:
# przygotowanie wektorów do trenowania
X_train = Train.drop(['device_id','group'], axis=1)
Y_train = Train['group']
# przygotowanie wektora testowego
X_test = Test.drop('device_id', axis=1)

In [42]:
# sprawdzenie poprawności macierzy
X_train.shape, Y_train.shape, X_test.shape

((74645, 105), (74645,), (112071, 105))

#### Nauka modelu algorytmem random forest 

In [46]:
# nauka modelu używając algorytmu regresji logistyczynej
logreg = LogisticRegression()
logreg.fit(X_train, Y_train)

# podstawowa ocena modelu na podstawie średniej liczby obserwacji pozytywnych do wszystkich
logreg.score(X_train, Y_train)

0.13944671444838905

In [47]:
# nauka modelu używają algorytmu random forest
randforest = RandomForestClassifier()
randforest.fit(X_train, Y_train)

# podstawowa ocena modelu na podstawie średniej liczby obserwacji pozytywnych do wszystkich
randforest.score(X_train, Y_train)

0.4445843659990622

In [48]:
# nauka modelu używają algorytmu ada boost
adaboost = AdaBoostClassifier()
adaboost.fit(X_train, Y_train)

# podstawowa ocena modelu na podstawie średniej liczby obserwacji pozytywnych do wszystkich
adaboost.score(X_train, Y_train)

0.15986335320517114

In [49]:
# nauka modelu używają algorytmu gradient boosting
gradientboost = GradientBoostingClassifier()
gradientboost.fit(X_train, Y_train)

# podstawowa ocena modelu na podstawie średniej liczby obserwacji pozytywnych do wszystkich
gradientboost.score(X_train, Y_train)

0.20065644048496215

In [ ]:
# nauka modelu używają algorytmu svc
svc = SVC()
svc.fit(X_train, Y_train)

# podstawowa ocena modelu na podstawie średniej liczby obserwacji pozytywnych do wszystkich
svc.score(X_train, Y_train)

In [ ]:
# nauka modelu używają algorytmu k najbliższych sąsiadów
kneigh = KNeighborsClassifier()
kneigh.fit(X_train, Y_train)

# podstawowa ocena modelu na podstawie średniej liczby obserwacji pozytywnych do wszystkich
kneigh.score(X_train, Y_train)